# Keras MNIST mlp

Dimension: 784x10<br>
Training size: 60,000<br>
Test size: 10,000

In [1]:
import keras as ks

Using Theano backend.


In [2]:
import numpy as np
from functools import reduce
import datetime

### Load data

In [3]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()
test_data = datahandler.load_testing()

In [4]:
train_image_array = np.asarray(train_data[0])
test_image_array = np.asarray(test_data[0])

In [5]:
train_label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    train_label_list.append(l)

train_label_array = np.asarray(train_label_list)

In [6]:
test_label_list = []
for i in test_data[1]:
    l = [0]*10
    l[i] = 1
    test_label_list.append(l)

test_label_array = np.asarray(test_label_list)

### Create and train model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD

In [8]:
model = Sequential([
    Dense(10, batch_input_shape=(None, 784), activation = 'sigmoid'),
    Dense(10, activation = 'softmax'),
])

In [20]:
sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics = ['accuracy'])

In [21]:
t = datetime.datetime.now()
hist = model.fit(train_image_array, 
                 train_label_array,
                 nb_epoch=10, 
                 batch_size=10)
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t))

Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.0950 - acc: 0.9702     
Epoch 2/10
  990/60000 [..............................] - ETA: 6s - loss: 0.0937 - acc: 0.9701

KeyboardInterrupt: 

### Make predictions

In [11]:
p = model.predict([test_image_array])

In [12]:
print("Dimensión predicción: {0}".format(p.shape)) #check prediction dimensions

Dimensión predicción: (10000, 10)


# Metrics

### Confusion matrix
Useful for metric calculation

In [13]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: test_label_array[j][i], range(0,10)))[0]
    conf_mtx[actual_label][max_pred] += 1
np.set_printoptions(suppress=True)
print(conf_mtx)

[[  923.     1.     8.    11.     2.    12.    19.     3.     1.     0.]
 [    0.  1108.     5.     8.     0.     0.     2.     1.    11.     0.]
 [   25.    25.   877.    16.    26.     5.    14.    18.    24.     2.]
 [   15.    11.    28.   883.     3.    27.     1.    15.    22.     5.]
 [    2.     5.     4.     3.   851.     4.    14.     0.     5.    94.]
 [   29.     4.    20.    80.    19.   587.    14.    13.   121.     5.]
 [   28.     4.    13.     0.    37.     9.   837.     3.    27.     0.]
 [   10.    40.    17.    26.    22.     0.     1.   838.     0.    74.]
 [   10.    80.    15.    37.    25.   133.    14.     9.   644.     7.]
 [   10.     4.     2.    20.    48.    24.     7.     9.     6.   879.]]


### Accuracy

In [14]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/10000)

In [15]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.84270


### Recall

In [16]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [17]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.94184
Recall 1: 0.97621
Recall 2: 0.84981
Recall 3: 0.87426
Recall 4: 0.86660
Recall 5: 0.65807
Recall 6: 0.87370
Recall 7: 0.81518
Recall 8: 0.66119
Recall 9: 0.87116

Recall mean: 0.83880


### Precision

In [18]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [19]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.87738
Precision 1: 0.86427
Precision 2: 0.88675
Precision 3: 0.81458
Precision 4: 0.82381
Precision 5: 0.73283
Precision 6: 0.90683
Precision 7: 0.92189
Precision 8: 0.74797
Precision 9: 0.82458

Precision mean: 0.84009
